<a href="https://colab.research.google.com/github/BlestxVentures/mattergen/blob/main/MatterGen_Test_for_github.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installation

1.   Clone the mattergen repo. It is best to have it forked, may need to make some changes
2.   Install prerequisites using uv
3.   Install git lfs
4.   Clone the remaining lfs files



In [ ]:
!git clone https://github.com/JoshDumo/mattergen.git

Cloning into 'mattergen'...
remote: Enumerating objects: 590, done.
remote: Counting objects: 100% (152/152), done.
remote: Compressing objects: 100% (66/66), done.
remote: Total 590 (delta 101), reused 86 (delta 86), pack-reused 438 (from 1)
Receiving objects: 100% (590/590), 1.60 MiB | 24.90 MiB/s, done.
Resolving deltas: 100% (248/248), done.


In [ ]:
%cd /content/mattergen

/content/mattergen


In [ ]:
!pip install uv
!uv venv .venv --python 3.10
!source .venv/bin/activate; uv pip install -e . --python .venv/bin/python

Streaming output truncated to the last 5000 lines.
   Building fire==0.7.0
   Building sentinels==1.0.0
⠼ Preparing packages... (209/244)
tzdata     ------------------------------ 254.80 KiB/339.69 KiB
jedi       ------------------------------ 718.43 KiB/1.50 MiB
ase        ------------------------------ 2.08 MiB/2.79 MiB
cryptography ------------------------------ 3.72 MiB/4.00 MiB
pillow     ------------------------------ 3.70 MiB/4.27 MiB
h5py       ------------------------------ 3.92 MiB/4.34 MiB
fonttools  ------------------------------ 3.74 MiB/4.39 MiB
pymatgen   ------------------------------ 3.33 MiB/4.71 MiB
sympy      ------------------------------ 3.15 MiB/5.90 MiB
babel      ------------------------------ 3.97 MiB/9.71 MiB
matplotlib ------------------------------ 3.97 MiB/11.07 MiB
jupyterlab ------------------------------ 3.80 MiB/11.14 MiB
pandas     ------------------------------ 2.74 MiB/12.46 MiB
nvidia-cuda-cupti-cu11 ------------------------------ 3.74 MiB/12.49 Mi

In [ ]:
!apt-get install nvidia-cuda-toolkit

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  fonts-dejavu-core fonts-dejavu-extra libaccinj64-11.5 libatk-wrapper-java libatk-wrapper-java-jni
  libbabeltrace1 libcub-dev libcublas11 libcublaslt11 libcudart11.0 libcufft10 libcufftw10
  libcuinj64-11.5 libcupti-dev libcupti-doc libcupti11.5 libcurand10 libcusolver11 libcusolvermg11
  libcusparse11 libdebuginfod-common libdebuginfod1 libegl-dev libfontenc1 libgail-common libgail18
  libgl-dev libgl1-mesa-dev libgles-dev libgles1 libglvnd-core-dev libglvnd-dev libglx-dev
  libgtk2.0-0 libgtk2.0-bin libgtk2.0-common libipt2 libnppc11 libnppial11 libnppicc11 libnppidei11
  libnppif11 libnppig11 libnppim11 libnppist11 libnppisu11 libnppitc11 libnpps11 libnvblas11
  libnvidia-compute-495 libnvidia-compute-510 libnvidia-compute-535 libnvidia-ml-dev libnvjpeg11
  libnvrtc-builtins11.5 libnvrtc11.2 libnvtoolsext1 libnvvm4 libopengl-dev libr

# Get necessary configs/checkpoints from GCS before use

In [ ]:
# In Colab, run this cell to authenticate with your Google account:
from google.colab import auth
auth.authenticate_user()

from google.cloud import storage

# Set your project ID
PROJECT_ID = 'phase-prediction'

# Initialize the Cloud Storage client; credentials are automatically picked up after authentication
client = storage.Client(project=PROJECT_ID)

# Example: List all buckets in your project
buckets = client.list_buckets()
for bucket in buckets:
    print(bucket.name)

In [ ]:
# get config file for azure_bulk_modulus (or other new configs)

!gsutil cp gs://mattergen-sc/property_configs/azure_bulk_modulus.yaml /content/mattergen/mattergen/conf/lightning_module/diffusion_module/model/property_embeddings

In [ ]:
# remove existing ckpt
!rm /content/mattergen/checkpoints/mattergen_base/checkpoints/last.ckpt

In [ ]:
#get base model checkpoint from GCS
!gsutil cp gs://mattergen-sc/model_checkpoints/base/last.ckpt /content/mattergen/checkpoints/mattergen_base/checkpoints/

***Before training azure_bulk_modulus etc. add the name of the exact name of the column in the dataset you want to train on to PROPERTY_SOURCE_IDS in /content/mattergen/mattergen/common/utils/globals.py***

# Generating Materials
## Unconditional generation


In [ ]:
!source .venv/bin/activate; export MODEL_PATH=mattergen_base; export RESULTS_PATH=results/ ; python mattergen/scripts/generate.py $RESULTS_PATH $MODEL_PATH --batch_size=2 --num_batches 1

MODELS_PROJECT_ROOT: /content/mattergen/mattergen
last.ckpt: 100% 461M/461M [00:03<00:00, 129MB/s]
config.yaml: 100% 5.49k/5.49k [00:00<00:00, 29.7MB/s]

Model config:
/content/mattergen/mattergen/common/utils/data_classes.py:95: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  with initialize_config_dir(str(self.model_path)):
auto_resume: true
checkpoint_path: null
data_module:
  _recursive_: true
  _target_: mattergen.common.data.datamodule.CrystDataModule
  average_density: 0.05771451654022283
  batch_size:
    train: 32
    val: 32
  max_epochs: 2200
  num_workers:
    train: 0
    val: 0
  properties:
  - dft_bulk_modulus
  - dft_band_gap
  - dft_mag_density
  - ml_bulk_modulus
  - hhi_score
  - space_group
  - energy_above_hull
  root_dir: datasets/cache/alex_mp_20/
  train_dataset:
    _target_: mattergen.common.data.dataset.CrystalDataset.from_cache_path
    cache_path: datas

In [ ]:
!zip -r /content/mattergen/results_unconditional.zip /content/mattergen/results

  adding: content/mattergen/results/ (stored 0%)
  adding: content/mattergen/results/generated_crystals.extxyz (deflated 57%)
  adding: content/mattergen/results/generated_trajectories.zip (stored 0%)
  adding: content/mattergen/results/generated_crystals_cif.zip (stored 0%)


In [ ]:
# dependencies for viewing atoms and extracting zip files
!pip install ase
from ase import Atoms
from ase.io import read, write
from ase.visualize import view
import zipfile
import os



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 28.4 MB/s eta 0:00:00


In [ ]:
zip_path = "/content/mattergen/results/generated_crystals_cif.zip"
extract_path = "/content/mattergen/gen_cif_files"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Extraction complete!")

cifs = os.listdir(extract_path+'/tmp')


Extraction complete!


In [ ]:
# select from list of cif files and look at one generated sample
atm = read(f'{extract_path}/tmp/{cifs[1]}')
view(atm, viewer='x3d')

In [ ]:
# to download the zip file of all of the cifs generated
from google.colab import files
files.download('/content/mattergen/results_unconditional.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Generate on a property

In [ ]:
!source .venv/bin/activate; export MODEL_NAME=dft_mag_density; export MODEL_PATH="checkpoints/$MODEL_NAME"; export RESULTS_PATH="results/$MODEL_NAME/" ; python mattergen/scripts/generate.py $RESULTS_PATH $MODEL_PATH --batch_size=16 --checkpoint_epoch=last --properties_to_condition_on="{'dft_mag_density': 0.15}" --diffusion_guidance_factor=2.0


MODELS_PROJECT_ROOT: /content/mattergen/mattergen
Traceback (most recent call last):
  File "/content/mattergen/mattergen/scripts/generate.py", line 104, in <module>
    _main()
  File "/content/mattergen/mattergen/scripts/generate.py", line 100, in _main
    fire.Fire(main)
  File "/content/mattergen/.venv/lib/python3.10/site-packages/fire/core.py", line 135, in Fire
    component_trace = _Fire(component, args, parsed_flag_args, context, name)
  File "/content/mattergen/.venv/lib/python3.10/site-packages/fire/core.py", line 468, in _Fire
    component, remaining_args = _CallAndUpdateTrace(
  File "/content/mattergen/.venv/lib/python3.10/site-packages/fire/core.py", line 684, in _CallAndUpdateTrace
    component = fn(*varargs, **kwargs)
  File "/content/mattergen/mattergen/scripts/generate.py", line 55, in main
    pretrained_name is not None or model_path is not None
AssertionError: Either pretrained_name or model_path must be provided.


In [ ]:
!ls -al results

total 42236
drwxr-xr-x  3 root root     4096 Jan 24 07:59 .
drwxr-xr-x 15 root root     4096 Jan 24 07:43 ..
drwxr-xr-x  2 root root     4096 Jan 24 07:57 dft_mag_density
-rw-r--r--  1 root root 21994867 Jan 24 07:59 dft_mag_density.zip
-rw-r--r--  1 root root    24565 Jan 24 07:41 generated_crystals_cif.zip
-rw-r--r--  1 root root    10662 Jan 24 07:41 generated_crystals.extxyz
-rw-r--r--  1 root root 21204569 Jan 24 07:42 generated_trajectories.zip


In [ ]:
!zip -r /content/mattergen/results/dft_mag_density.zip /content/mattergen/results/dft_mag_density

  adding: content/mattergen/results/dft_mag_density/ (stored 0%)
  adding: content/mattergen/results/dft_mag_density/generated_crystals.extxyz (deflated 63%)
  adding: content/mattergen/results/dft_mag_density/generated_trajectories.zip (stored 0%)
  adding: content/mattergen/results/dft_mag_density/generated_crystals_cif.zip (stored 0%)


In [ ]:
files.download('/content/mattergen/results/dft_mag_density.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Generate on multiple conditions

In [ ]:
!source .venv/bin/activate; export MODEL_NAME=chemical_system_energy_above_hull; export MODEL_PATH="checkpoints/$MODEL_NAME"; export RESULTS_PATH="results/$MODEL_NAME/" ; python mattergen/scripts/generate.py $RESULTS_PATH $MODEL_PATH --batch_size=16 --checkpoint_epoch=last --properties_to_condition_on="{'energy_above_hull': 0.05, 'chemical_system': 'Li-O'}" --diffusion_guidance_factor=2.0


MODELS_PROJECT_ROOT: /content/mattergen/mattergen
Traceback (most recent call last):
  File "/content/mattergen/.venv/lib/python3.10/site-packages/huggingface_hub/utils/_http.py", line 409, in hf_raise_for_status
    response.raise_for_status()
  File "/content/mattergen/.venv/lib/python3.10/site-packages/requests/models.py", line 1024, in raise_for_status
    raise HTTPError(http_error_msg, response=self)
requests.exceptions.HTTPError: 404 Client Error: Not Found for url: https://huggingface.co/microsoft/mattergen/resolve/main/checkpoints/checkpoints/chemical_system_energy_above_hull/checkpoints/last.ckpt

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/content/mattergen/mattergen/scripts/generate.py", line 104, in <module>
    _main()
  File "/content/mattergen/mattergen/scripts/generate.py", line 100, in _main
    fire.Fire(main)
  File "/content/mattergen/.venv/lib/python3.10/site-packages/fire/core.py", line 135, in

In [ ]:
!ls -al results

total 63836
drwxr-xr-x  4 root root     4096 Jan 24 08:17 .
drwxr-xr-x 15 root root     4096 Jan 24 07:43 ..
drwxr-xr-x  2 root root     4096 Jan 24 08:16 chemical_system_energy_above_hull
-rw-r--r--  1 root root 22111111 Jan 24 08:17 chemical_system_energy_above_hull.zip
drwxr-xr-x  2 root root     4096 Jan 24 07:57 dft_mag_density
-rw-r--r--  1 root root 21994867 Jan 24 07:59 dft_mag_density.zip
-rw-r--r--  1 root root    24565 Jan 24 07:41 generated_crystals_cif.zip
-rw-r--r--  1 root root    10662 Jan 24 07:41 generated_crystals.extxyz
-rw-r--r--  1 root root 21204569 Jan 24 07:42 generated_trajectories.zip


In [ ]:
!zip -r /content/mattergen/results/chemical_system_energy_above_hull.zip /content/mattergen/results/chemical_system_energy_above_hull

  adding: content/mattergen/results/chemical_system_energy_above_hull/ (stored 0%)
  adding: content/mattergen/results/chemical_system_energy_above_hull/generated_crystals.extxyz (deflated 64%)
  adding: content/mattergen/results/chemical_system_energy_above_hull/generated_trajectories.zip (stored 0%)
  adding: content/mattergen/results/chemical_system_energy_above_hull/generated_crystals_cif.zip (stored 0%)


In [ ]:
files.download('/content/mattergen/results/chemical_system_energy_above_hull.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Evaluation

In [ ]:
!source .venv/bin/activate; export RESULTS_PATH=results/ ; python mattergen/scripts/evaluate.py --structures_path=$RESULTS_PATH --relax=True --structure_matcher='disordered' --save_as="$RESULTS_PATH/metrics.json"

MODELS_PROJECT_ROOT: /content/mattergen/mattergen
2025-03-24 20:21:06.929 | INFO     | mattersim.forcefield.potential:from_checkpoint:877 - Loading the pre-trained mattersim-v1.0.0-1M.pth model
0it [00:00, ?it/s]Traceback (most recent call last):
  File "/content/mattergen/mattergen/scripts/evaluate.py", line 55, in <module>
    _main()
  File "/content/mattergen/mattergen/scripts/evaluate.py", line 51, in _main
    fire.Fire(main)
  File "/content/mattergen/.venv/lib/python3.10/site-packages/fire/core.py", line 135, in Fire
    component_trace = _Fire(component, args, parsed_flag_args, context, name)
  File "/content/mattergen/.venv/lib/python3.10/site-packages/fire/core.py", line 468, in _Fire
    component, remaining_args = _CallAndUpdateTrace(
  File "/content/mattergen/.venv/lib/python3.10/site-packages/fire/core.py", line 684, in _CallAndUpdateTrace
    component = fn(*varargs, **kwargs)
  File "/content/mattergen/mattergen/scripts/evaluate.py", line 38, in main
    metrics = eva

In [ ]:
!ls -al results/

total 63840
drwxr-xr-x  4 root root     4096 Jan 24 08:30 .
drwxr-xr-x 15 root root     4096 Jan 24 07:43 ..
drwxr-xr-x  2 root root     4096 Jan 24 08:16 chemical_system_energy_above_hull
-rw-r--r--  1 root root 22111111 Jan 24 08:17 chemical_system_energy_above_hull.zip
drwxr-xr-x  2 root root     4096 Jan 24 07:57 dft_mag_density
-rw-r--r--  1 root root 21994867 Jan 24 07:59 dft_mag_density.zip
-rw-r--r--  1 root root    24565 Jan 24 07:41 generated_crystals_cif.zip
-rw-r--r--  1 root root    10662 Jan 24 07:41 generated_crystals.extxyz
-rw-r--r--  1 root root 21204569 Jan 24 07:42 generated_trajectories.zip
-rw-r--r--  1 root root     2823 Jan 24 08:30 metrics.json


In [ ]:
files.download('/content/mattergen/results/metrics.json')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Training

### Get data from GCS

In [ ]:
import pandas as pd

In [ ]:
#!csv-to-dataset --csv-folder datasets/mp_20/ --dataset-name mp_20 --cache-folder datasets/cache

In [ ]:
# In Colab, run this cell to authenticate with your Google account:
from google.colab import auth
auth.authenticate_user()

from google.cloud import storage

# Set your project ID
PROJECT_ID = 'phase-prediction'

# Initialize the Cloud Storage client; credentials are automatically picked up after authentication
client = storage.Client(project=PROJECT_ID)

# Example: List all buckets in your project
buckets = client.list_buckets()
for bucket in buckets:
    print(bucket.name)

mattergen-sc
phase-prediction
phase-prediction_cloudbuild


In [ ]:
!mkdir /content/zipped_mg_datasets

In [ ]:
!gsutil cp gs://mattergen-sc/zipped_mg_datasets/*.zip /content/zipped_mg_datasets

Copying gs://mattergen-sc/zipped_mg_datasets/alex_mp_20.zip...
==> NOTE: You are downloading one or more large file(s), which would
run significantly faster if you enabled sliced object downloads. This
feature is enabled by default but requires that compiled crcmod be
installed (see "gsutil help crcmod").

Copying gs://mattergen-sc/zipped_mg_datasets/cifs.zip...
Copying gs://mattergen-sc/zipped_mg_datasets/mp_20.zip...
/ [3 files][190.2 MiB/190.2 MiB]                                                
Operation completed over 3 objects/190.2 MiB.                                    


In [ ]:
!unzip /content/zipped_mg_datasets/mp_20.zip -d /content/mattergen/datasets


Archive:  /content/zipped_mg_datasets/mp_20.zip
   creating: /content/mattergen/datasets/mp_20/
  inflating: /content/mattergen/datasets/mp_20/val.csv  
  inflating: /content/mattergen/datasets/mp_20/test.csv  
  inflating: /content/mattergen/datasets/mp_20/train.csv  


In [ ]:
mp20 = pd.read_csv('/content/mattergen/datasets/mp_20/train.csv')
mp20.head()

,Unnamed: 0,material_id,formation_energy_per_atom,dft_band_gap,pretty_formula,e_above_hull,elements,cif,spacegroup_number,azure_bulk_modulus,larsen_score_2d,Si_100_mismatch,azure_band_gap,dft_bulk_modulus,dft_poisson_ratio,dft_mag_density
0,0,mp-1221227,-1.637460,0.2133,Na3MnCoNiO6,0.043001,"['Co', 'Mn', 'Na', 'Ni', 'O']",# generated using pymatgen\r\ndata_Na3MnCoNiO6...,8.0,100.027488,0.000000e+00,0.014483,0.576968,NaN,NaN,7.068804e-02
1,1,mp-974729,-0.314759,0.0000,Nd(Al2Cu)4,0.000000,"['Al', 'Cu', 'Nd']",# generated using pymatgen\r\ndata_Nd(Al2Cu)4\...,139.0,94.080115,0.000000e+00,0.023284,0.000028,NaN,NaN,2.025358e-09
2,2,mp-1185360,-0.193761,0.0000,LiMnIr2,0.018075,"['Ir', 'Li', 'Mn']",# generated using pymatgen\r\ndata_LiMnIr2\r\n...,225.0,206.381562,0.000000e+00,0.001716,0.000003,NaN,NaN,4.048347e-02
3,3,mp-1188861,-0.584694,3.8556,LiCSN,0.048847,"['C', 'Li', 'N', 'S']",# generated using pymatgen\r\ndata_LiCSN\r\n_s...,62.0,24.972078,1.221245e-15,0.002367,3.701200,NaN,NaN,1.194041e-06
4,4,mp-677272,-2.474759,0.4707,La2EuS4,0.000000,"['Eu', 'La', 'S']",# generated using pymatgen\r\ndata_La2EuS4\r\n...,122.0,65.992807,0.000000e+00,0.011328,0.480415,NaN,NaN,6.011521e-03


In [ ]:
!source .venv/bin/activate; python mattergen/scripts/csv_to_dataset.py --csv-folder datasets/mp_20/ --dataset-name mp_20 --cache-folder datasets/cache

MODELS_PROJECT_ROOT: /content/mattergen/mattergen
Processing datasets/mp_20//test.csv
Parsing CIFs:  76% 6906/9046 [00:32<00:10, 213.85it/s]/content/mattergen/.venv/lib/python3.10/site-packages/pymatgen/core/periodic_table.py:289: UserWarning: No Pauling electronegativity for Ar. Setting to NaN. This has no physical meaning, and is mainly done to avoid errors caused by the code expecting a float.
  warnings.warn(
Parsing CIFs:  99% 8931/9046 [00:42<00:00, 204.92it/s]/content/mattergen/.venv/lib/python3.10/site-packages/pymatgen/core/periodic_table.py:289: UserWarning: No Pauling electronegativity for He. Setting to NaN. This has no physical meaning, and is mainly done to avoid errors caused by the code expecting a float.
  warnings.warn(
Parsing CIFs: 100% 9046/9046 [00:42<00:00, 212.18it/s]
Converting structures to numpy: 100% 9046/9046 [00:13<00:00, 664.40it/s]
Storing cached dataset in datasets/cache/mp_20/test.
Processing datasets/mp_20//train.csv
Parsing CIFs:  62% 16785/27136 [01

In [ ]:
# make a json file for the bulk modulus property with correct format in each cache folder using the

import json

# Train data
train = pd.read_csv('/content/mattergen/datasets/mp_20/train.csv')
train_out = '/content/mattergen/datasets/cache/mp_20/train/azure_bulk_modulus.json'

train_data = {
    "values": train['azure_bulk_modulus'].values.tolist(),
    "property_source_doc_id": ["azure_bulk_modulus"] * len(train['azure_bulk_modulus'].values.tolist())
}

with open(train_out, 'w') as f:
    json.dump(train_data, f)

# Test data

test = pd.read_csv('/content/mattergen/datasets/mp_20/test.csv')
test_out = '/content/mattergen/datasets/cache/mp_20/test/azure_bulk_modulus.json'
test_data = {
    "values": test['azure_bulk_modulus'].values.tolist(),
    "property_source_doc_id": ["azure_bulk_modulus"] * len(test['azure_bulk_modulus'].values.tolist())
}

with open(test_out, 'w') as f:
    json.dump(test_data, f)


# val data
val = pd.read_csv('/content/mattergen/datasets/mp_20/val.csv')
val_out = '/content/mattergen/datasets/cache/mp_20/val/azure_bulk_modulus.json'

val_data = {
    "values": val['azure_bulk_modulus'].values.tolist(),
    "property_source_doc_id": ["azure_bulk_modulus"] * len(val['azure_bulk_modulus'].values.tolist())
}

with open(val_out, 'w') as f:
    json.dump(val_data, f)


In [ ]:
pd.read_json('/content/mattergen/datasets/cache/mp_20/test/azure_bulk_modulus.json')

,values,property_source_doc_id
0,15.080567,azure_bulk_modulus
1,130.800381,azure_bulk_modulus
2,166.812766,azure_bulk_modulus
3,42.691927,azure_bulk_modulus
4,96.378471,azure_bulk_modulus
...,...,...
9041,120.043028,azure_bulk_modulus
9042,57.473123,azure_bulk_modulus
9043,107.193811,azure_bulk_modulus
9044,122.325035,azure_bulk_modulus


## Train from scratch

In [ ]:
!source .venv/bin/activate; python mattergen/scripts/csv_to_dataset.py --csv-folder datasets/mp_20/ --dataset-name mp_20 --cache-folder datasets/cache

MODELS_PROJECT_ROOT: /content/mattergen/mattergen
Processing datasets/mp_20//train.csv
Parsing CIFs:  62% 16795/27136 [02:21<01:11, 143.92it/s]/content/mattergen/.venv/lib/python3.10/site-packages/pymatgen/core/periodic_table.py:289: UserWarning: No Pauling electronegativity for He. Setting to NaN. This has no physical meaning, and is mainly done to avoid errors caused by the code expecting a float.
  warnings.warn(
Parsing CIFs: 100% 27136/27136 [03:42<00:00, 121.80it/s]
Converting structures to numpy:   7% 1868/27136 [00:03<00:51, 488.88it/s]
Traceback (most recent call last):
  File "/content/mattergen/mattergen/scripts/csv_to_dataset.py", line 43, in <module>
    main()
  File "/content/mattergen/mattergen/scripts/csv_to_dataset.py", line 36, in main
    CrystalDataset.from_csv(
  File "/content/mattergen/mattergen/common/data/dataset.py", line 172, in from_csv
    return CrystalDatasetBuilder.from_csv(
  File "/content/mattergen/mattergen/common/data/dataset.py", line 540, in from

In [ ]:
!source .venv/bin/activate; python scripts/run.py data_module=mp_20 ~trainer.logger trainer.accumulate_grad_batches=32

MODELS_PROJECT_ROOT: /content/mattergen/mattergen
/content/mattergen/.venv/lib/python3.10/site-packages/hydra/_internal/defaults_list.py:251: UserWarning: In 'default': Defaults list is missing `_self_`. See https://hydra.cc/docs/1.2/upgrades/1.0_to_1.1/default_composition_order for more information
  warnings.warn(msg, UserWarning)
/content/mattergen/.venv/lib/python3.10/site-packages/hydra/_internal/hydra.py:119: UserWarning: Future Hydra versions will no longer change working directory at job runtime by default.
See https://hydra.cc/docs/1.2/upgrades/1.1_to_1.2/changes_to_job_working_dir/ for more information.
  ret = run_job(
data_module:
  _target_: mattergen.common.data.datamodule.CrystDataModule
  _recursive_: true
  properties: []
  transforms:
  - _target_: mattergen.common.data.transform.symmetrize_lattice
    _partial_: true
  - _target_: mattergen.common.data.transform.set_chemical_system_string
    _partial_: true
  dataset_transforms:
  - _target_: mattergen.common.data.d

## Finetune on property

In [ ]:
!source .venv/bin/activate; export PROPERTY=dft_bulk_modulus; export MODEL_PATH=checkpoints/mattergen_base; python mattergen/scripts/finetune.py adapter.model_path=$MODEL_PATH data_module=mp_20 +lightning_module/diffusion_module/model/property_embeddings@adapter.adapter.property_embeddings_adapt.$PROPERTY=$PROPERTY ~trainer.logger data_module.properties=["$PROPERTY"]


MODELS_PROJECT_ROOT: /content/mattergen/mattergen
/content/mattergen/.venv/lib/python3.10/site-packages/hydra/_internal/defaults_list.py:251: UserWarning: In 'finetune': Defaults list is missing `_self_`. See https://hydra.cc/docs/1.2/upgrades/1.0_to_1.1/default_composition_order for more information
  warnings.warn(msg, UserWarning)
/content/mattergen/.venv/lib/python3.10/site-packages/hydra/_internal/hydra.py:119: UserWarning: Future Hydra versions will no longer change working directory at job runtime by default.
See https://hydra.cc/docs/1.2/upgrades/1.1_to_1.2/changes_to_job_working_dir/ for more information.
  ret = run_job(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/content/mattergen/.venv/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/logger_connector/logger_connector.py:67: UserWarning: Starting from v1.9.0, `tensorboardX` has been removed as a depen

In [ ]:
!source .venv/bin/activate; export PROPERTY=azure_bulk_modulus; export MODEL_PATH=checkpoints/mattergen_base; python mattergen/scripts/finetune.py adapter.model_path=$MODEL_PATH data_module=mp_20 +lightning_module/diffusion_module/model/property_embeddings@adapter.adapter.property_embeddings_adapt.$PROPERTY=$PROPERTY ~trainer.logger data_module.properties=["$PROPERTY"]

MODELS_PROJECT_ROOT: /content/mattergen/mattergen
/content/mattergen/.venv/lib/python3.10/site-packages/hydra/_internal/defaults_list.py:251: UserWarning: In 'finetune': Defaults list is missing `_self_`. See https://hydra.cc/docs/1.2/upgrades/1.0_to_1.1/default_composition_order for more information
  warnings.warn(msg, UserWarning)
/content/mattergen/.venv/lib/python3.10/site-packages/hydra/_internal/hydra.py:119: UserWarning: Future Hydra versions will no longer change working directory at job runtime by default.
See https://hydra.cc/docs/1.2/upgrades/1.1_to_1.2/changes_to_job_working_dir/ for more information.
  ret = run_job(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/content/mattergen/.venv/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/logger_connector/logger_connector.py:67: UserWarning: Starting from v1.9.0, `tensorboardX` has been removed as a depen

### Generating using fined-tuned base model -- conditoned

### Finetune on multiple properties